# Maryland Child and Dependent Care Credit

Maryland's Child and Dependent Care Tax Credit (CDCC) provides up to 32% of the federal CDCC, depending on income and filing status.

## Examples

Consider a single parent in Maryland with one child and \$8,000 child care expenses (the maximum for one child in 2021).
Since the federal CDCC changed in 2021 (2022 returns to 2020 law), this chart shows all three years via the slider.
The example is limited to single parents because the federal CDCC is capped by the lower of a head and spouse's earnings.

In [4]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px


def make_cdcc(children, year):
    sim = IndividualSim(year=year)
    sim.add_person(name="head", is_tax_unit_head=True)
    members = ["head"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=5)
        members += [child]
    sim.add_tax_unit(
        name="tax_unit",
        members=members,
        tax_unit_childcare_expenses=8_000 * children,
    )
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MD")
    sim.vary("employment_income", max=100_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            md_cdcc=sim.calc("md_cdcc")[0].round(),
            cdcc_mtr=-sim.deriv(
                "md_cdcc",
                "employment_income",
                wrt_target="head",
            ),
            year=year,
            children=children,
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for year in [2020, 2021, 2022]:
    for children in range(1, 3):
        l.append(make_cdcc(children, year))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    md_cdcc="MD CDCC",
    scb_mtr="MD CDCC Marginal Tax Rate",
    year="Year",
    children="Children",
)

fig = px.line(
    df,
    "employment_income",
    "md_cdcc",
    color="children",
    animation_frame="year",
    labels=LABELS,
    title="Maryland CDCC",
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, df.md_cdcc.max() * 1.05],
)
fig.show()

The Maryland CDCC phases in at 11% of income, then phases out in a stepped way that produces many small cliffs (infinite marginal tax rates).

In [5]:
fig = px.line(
    df,
    "employment_income",
    "cdcc_mtr",
    color="children",
    animation_frame="year",
    labels=LABELS,
    title="Maryland CDCC marginal tax rate",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".1%", yaxis_range=[-0.2, 0.2]
)
fig.show()

## Budgetary impact

Applying 2022 rules to the 2020 Current Population Survey, PolicyEngine US estimates that the Maryland CDCC costs \$13 million.

In [9]:
from policyengine_us import Microsimulation

sim = Microsimulation(dataset_year=2020)

sim.calc("md_cdcc", period=2022).sum() / 1e6

13.179921692003392